In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import glob
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
from time import time
from torchsummary import summary
import math

In [2]:
test_size1 = 0.2
test_size2 = 0.5
batch_size = 64
seed = 0
lr = 0.1
first_layer_neurons = 1000
second_layer_neurons = 100
torch.manual_seed(seed)

In [3]:
root = './Data/'
classes = ('English', 'Mandarin', 'Persian')

# Get dataset
image_data = torchvision.datasets.ImageFolder(root=root, transform=transforms.ToTensor())

# one hot encoding
le = LabelEncoder()
class_data = le.fit_transform(classes)
oneh_encoder = OneHotEncoder()
class_onehot = oneh_encoder.fit_transform(class_data.reshape(-1, 1)).toarray()

# dataset splitting
train_data, valid_test_data = train_test_split(image_data, test_size=test_size1, random_state=seed)
valid_data, test_data = train_test_split(valid_test_data, test_size=test_size2, random_state=seed)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './Data/'

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*248, first_layer_neurons)
        self.fc2 = nn.Linear(first_layer_neurons, second_layer_neurons)
        self.fc3 = nn.Linear(first_layer_neurons, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2)
        x = self.fc3(x)
        return x

def accuracy(model,loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct/total

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
criterion = nn.MSELoss()
net = Net()
optimizer = torch.optim.SGD(net.parameters(),lr)

In [ ]:
trainLossRec = []
validLossRec = []
testLossRec = []
trainAccRec = []
validAccRec = []
testAccRec = []
nRec = []
startTime = time()

for epoch in range(numepoch):
    running_loss = 0.0
    numbatch = 0
    for i, train_data in enumerate(train_loader, 0):
        inputs, labels = train_data
        optimizer.zero_grad()
        predictT = net(inputs)
        lossT= loss_fcn(input=predictT, target=torch.Tensor(class_onehot[labels]))
        lossT.backward()
        optimizer.step()
        running_loss += lossT.item()
        numbatch += 1
    trainLossRec.append(running_loss/numbatch)
    accuT = accuracy(net,train_loader)
    trainAccRec.append(accuT)
    
    running_loss = 0.0
    numbatch = 0
    for i, valid_data in enumerate(valid_loader):
        inputs, labels = valid_data
        predictV = net(inputs.float())
        lossV = loss_fcn(input=predictV, target=torch.Tensor(class_onehot[labels]))
        running_loss += lossV.item()
        numbatch += 1
    validLossRec.append(running_loss/numbatch)
    accuV = accuracy(net,valid_loader)
    validAccRec.append(accuV)
    
    nRec.append(epoch)
    print("Epoch:",nRec[-1]+1,"Training Accuracy:",accuT,"Validation Accuracy:",accuV)

for i, test_data in enumerate(test_loader
        inputs, labels = test_data
        predict = net(inputs.float())
        loss = loss_fcn(input=predict, target=torch.Tensor(class_onehot[labels]))
        running_loss += lossV.item()
        numbatch += 1
    testLossRec.append(running_loss/numbatch)
    accu = accuracy(net,valid_loader)
    testAccRec.append(accu)
                              
timeElapsed = time()-startTime

print('Training Time:',timeElapsed)

In [ ]:
# PLOT
plt.plot(nRec,trainLossRec, label='Train')
plt.plot(nRec,validLossRec, label='Validation')
plt.title("Loss vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.plot(nRec,trainAccRec, label='Train')
plt.plot(nRec,validAccRec, label='Validation')
plt.title("Accuracy vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()